In [ ]:
!pip install transformers 

In [ ]:
!pip install gradio 
!pip install happytransformer

In [ ]:
## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.13.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages 
"""


Remember to restart the runtime for the kernel to pick up any upgraded packages 


In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_1024")

In [8]:
from happytransformer import HappyTextToText, TTSettings
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Aditya02/Speech_Analyzer_Model")

model = AutoModelForTokenClassification.from_pretrained("Aditya02/Speech_Analyzer_Model", from_tf = True)

All TF 2.0 model weights were used when initializing DistilBertForTokenClassification.

All the weights of DistilBertForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForTokenClassification for predictions without further training.


In [14]:
from transformers import pipeline
classifier = pipeline("ner",model = model, tokenizer = tokenizer)

In [15]:
def transcribe(audio):
  """Speech to text using Nvidia Nemo"""
  text = asr_model.transcribe(paths2audio_files=[audio])[0]
  #Disfluency Detection
  output = classifier(text)
  # Add the prefix "grammar: " before each input 
  correct = happy_tt.generate_text("grammar: " + text, args=args)
  return text, {"text": text, "entities": output},  correct.text 

In [25]:
import gradio as gr
gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(source="microphone", type="filepath"),
    ],
    outputs=[
        "textbox",
        gr.HighlightedText(combine_adjacent = True,color_map = {'Fluent':'green','Disfluent':'red'},label = "Disfluencies in the text"),
        "textbox"

    ]).launch()

[NeMo W 2023-01-18 09:01:38 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/gradio/components.py:3370: UserWarning: The 'color_map' parameter has been moved from the constructor to `HighlightedText.style()` 
      warnings.warn(
    


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>